In [83]:
#Support
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
#Data
data = pd.read_csv("file://localhost/users/nicki/desktop/udvikling/python/data/melb_data.csv",parse_dates=True)

DATA
-

In [84]:
# Select target
y = data.Price

# Removing objects
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Dividing into training and validation
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

In [85]:
#Shape for seeing rows and colums
print(X_train.shape)
# See colums with na + count numbers of na
print(X_train.isnull().sum()[X_train.isnull().sum() > 0])
# See some general statistics
#print(X_train.describe())

(10864, 12)
Car               52
BuildingArea    5193
YearBuilt       4312
dtype: int64


ADDRESSING NA VALUES
-

In [86]:
# Function for comparing different approaches
from sklearn.ensemble import RandomForestRegressor
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

Method 1: Drop columns
-

In [87]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

In [88]:
#Shape and missing values
print(reduced_X_train.shape)
print(reduced_X_train.isnull().sum()[reduced_X_train.isnull().sum() > 0])
#we have 11k rows with 9 features and no missing values

(10864, 9)
Series([], dtype: int64)


In [89]:
#Testing MAE
print("MAE from Drop columns with missing values:")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Drop columns with missing values:
186029.25597692686


Method 2: Simple Imputation
-

In [90]:
from sklearn.impute import SimpleImputer
# Imputation  strategy mean, median, most_frequent or constant (last 2 works with strings too)
my_imputer = SimpleImputer(strategy="mean")
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


In [91]:
#Checking data
print(imputed_X_train.shape)
print(imputed_X_train.isnull().sum()[imputed_X_train.isnull().sum() > 0])
#we have 11k rows with 12 features and no missing values

(10864, 12)
Series([], dtype: int64)


In [92]:
print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))
# gives a bit better results than just dropping colums

MAE from Approach 2 (Imputation):
175503.57803141873


Method 2.5: Simple Imputation + binary traction
-

In [93]:
# Make copy to avoid changing original data (when adding new columns)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()


In [94]:
from sklearn.impute import SimpleImputer
# Imputation alternative is add_indicator=true (but it will still need column names)
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

In [95]:
#Checking data
print(imputed_X_train_plus.shape)
print(imputed_X_train_plus.isnull().sum()[imputed_X_train_plus.isnull().sum() > 0])
#3 new columns for the 3 columns which had missing values - its correct!

(10864, 15)
Series([], dtype: int64)


In [96]:
#Testing MAE
print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))
#it shows reduced MAE compared to 

MAE from Approach 3 (An Extension to Imputation):
176964.14620765834
